In [1]:
import real_estate_api as rea

from typing import Union, List
from types import ModuleType
from beartype import beartype
import warnings

import numpy as np
import pandas as pd
import sys
import importlib

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import keys as k
keys = k.getKeys()

import pgeocode

'''
### For pgeocode to work, I am not going to pretend to know what this does ###
# https://stackoverflow.com/questions/27835619/urllib-and-ssl-certificate-verify-failed-error
import certifi
import os
import ssl

openssl_dir, openssl_cafile = os.path.split(
        ssl.get_default_verify_paths().openssl_cafile
)

os.chdir(openssl_dir)
relpath_to_certifi_cafile = os.path.relpath(certifi.where())

try:
    os.remove(openssl_cafile)
except FileNotFoundError:
    pass

os.symlink(relpath_to_certifi_cafile, openssl_cafile)
###
'''

'\n### For pgeocode to work, I am not going to pretend to know what this does ###\n# https://stackoverflow.com/questions/27835619/urllib-and-ssl-certificate-verify-failed-error\nimport certifi\nimport os\nimport ssl\n\nopenssl_dir, openssl_cafile = os.path.split(\n        ssl.get_default_verify_paths().openssl_cafile\n)\n\nos.chdir(openssl_dir)\nrelpath_to_certifi_cafile = os.path.relpath(certifi.where())\n\ntry:\n    os.remove(openssl_cafile)\nexcept FileNotFoundError:\n    pass\n\nos.symlink(relpath_to_certifi_cafile, openssl_cafile)\n###\n'

In [2]:
user_home = rea.get_UserHome(keys['SampleHouse'].strip("\'"))
hoi = rea.get_HousesOfInterest(user_home, n=1000, listed_to_sold_ratio=0.3, verbose=True)
gd = rea.geo_data(hoi['geo'])

In [3]:
# Can this be the first step in the pipeline or no?
fg = rea.FeatureGenerator(
    houses = [rea.house(h) for h in hoi['houses']], 
    gd=gd,
    user_home=user_home
)

In [4]:
# Basic transformer to dataframe.
class ToDataFrame(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return pd.DataFrame(X)

In [5]:
pipe = Pipeline([
    ('format_data', ToDataFrame())
])

output = pipe.fit_transform(fg.features, fg.targets)

In [6]:
to_data_frame = ToDataFrame()
X_t = to_data_frame.fit_transform(fg.features)
X_t

,Days_listed,Days_updated,baths_full,baths_3qtr,baths_half,baths_1qtr,year_built,lot_sqft,sqft,garage,stories,beds,type,tags,new_construction,distance_to_home
0,0,0,1,1,0,0,1942,7000,1992,1,1,4,single_family,"[community_outdoor_space, den_or_office, dinin...",False,0.087471
1,0,0,1,0,1,0,1988,1071,1300,0,1,2,single_family,"[city_view, community_outdoor_space, community...",False,0.101667
2,0,0,1,1,0,0,1913,3500,1750,0,1,3,single_family,"[community_security_features, den_or_office, d...",False,0.205025
3,0,0,1,1,1,0,1980,8100,3280,2,1,3,single_family,"[community_outdoor_space, dining_room, dishwas...",False,0.068888
4,0,0,1,1,1,0,1948,7680,2180,2,1,4,single_family,"[central_air, community_outdoor_space, dining_...",False,0.083503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,73,1,1,1,0,1906,4000,2260,1,1,3,single_family,"[city_view, community_outdoor_space, den_or_of...",False,0.091754
996,0,72,3,0,1,0,2006,3600,3340,2,1,5,single_family,"[central_air, community_outdoor_space, communi...",False,0.127658
997,0,71,1,1,0,0,1952,5800,2360,1,1,2,single_family,"[community_outdoor_space, community_security_f...",False,0.080205
998,0,71,1,1,0,0,1906,5700,2880,1,1,2,single_family,"[community_outdoor_space, dining_room, dishwas...",False,0.093157


In [7]:
X_t['targets'] = fg.targets

In [16]:
X_t[X_t['targets'] < 200000]

,Days_listed,Days_updated,baths_full,baths_3qtr,baths_half,baths_1qtr,year_built,lot_sqft,sqft,garage,stories,beds,type,tags,new_construction,distance_to_home,targets
117,32,33,0,0,0,0,1970,1306800,0,0,1,0,single_family,"[hill_or_mountain_view, lake_view, view, water...",False,0.132380,130000
411,0,19,1,0,0,0,1985,10890000,250,0,1,1,single_family,"[city_view, community_boat_facilities, communi...",False,0.101667,159000
428,0,0,1,0,0,0,1929,1500,950,0,1,3,single_family,[],False,0.147240,167691
760,0,0,1,1,0,0,1946,6300,770,0,1,2,single_family,[],False,0.069057,170000
